### Create n_gram for each project and split for each sentiment  ( use for finding important features for each sentiment&project )

In [1]:
# prepare file for ngweight
import csv
import os
texts = dict()
project_names = list()
def read_csv():
    with open('dataset/dataset.csv', 'r',encoding='latin1') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['project'] not in texts:
                texts[row['project']] = dict()
                project_names.append(row['project'])
            if row['oracle'] not in texts[row['project']]:
                texts[row['project']][row['oracle']] = list()
            texts[row['project']][row['oracle']].append(row['text'])
            
def write_csv():
    if not os.path.exists('dataset/importance_features/'):
        os.makedirs('dataset/importance_features/')
    for project_name in project_names:
        for label in texts[project_name]:
            with open('dataset/importance_features/'+project_name+'_'+label+'_n_gram_prep','w') as file:
                for row in texts[project_name][label]:
                    file.write(chr(0x02)+chr(0x03)+"\n")
                    file.write(row+"\n")
read_csv()
print(len(texts))
write_csv()


3


In [2]:
from subprocess import call
# call("../ngweight-master/waf",shell=True)
for project_name in project_names:
    for label in texts[project_name]:
        call("../ngweight-master/bin/default/ngweight -w -s 0 < dataset/importance_features/"+project_name+'_'+label+"_n_gram_prep > dataset/importance_features/"+project_name+'_'+label+"_n_gram",shell=True)
    

In [3]:
import csv
import sys

for project_name in project_names:
    for label in texts[project_name]:
        n_gram_filter = list()
        with open("dataset/importance_features/"+project_name+'_'+label+"_n_gram") as csvfile:
            reader = csv.reader(csvfile, delimiter='\t', quotechar='|')
            for row in reader:
                if int(row[2]) <= 1:
                    continue
                n_gram_filter.append(row)

        with open("dataset/importance_features/"+project_name+'_'+label+"_n_gram_filter",'w') as csvfile:
            writer = csv.writer(csvfile,delimiter='\t', quotechar='|')
            writer.writerows(n_gram_filter)
